In [1]:
cd /Users/karolinegriesbach/Documents/Innkeepr/Git/evaluation-and-execution-scripts/

In [2]:
import logging
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from general_functions.datetime_helper import transform_date_to_timestamp_milliseconds
from general_functions.return_account_ids import return_account_ids
from general_functions.constants import return_api_url
from general_functions.call_api_with_account_id import call_api_with_accountId, send_to_innkeepr_api_paginated

In [3]:
customer = "Junglueck"
goal_name = ["checkout_completed"] #"checkout_completed"
url = return_api_url()
print(f"url = {url}")
account_id = return_account_ids()
account_id = [acc["id"] for acc in account_id if acc["name"] == customer]
account_id = account_id[0]
data_path = "/Users/karolinegriesbach/Documents/Innkeepr/Git/innkeepr-analytics/"
#file_name = "adding_conversion.csv"
file_name = "cate_meta_synthetic_conversions.csv"
file_name = "test_data.csv"

# Load Conversions

In [4]:
data = pd.read_csv(f"{data_path}{file_name}")
data_max_date = data["created"].max()
from_date = pd.to_datetime(data_max_date).strftime("%Y-%m-%d")
to_date = (pd.to_datetime(data_max_date)+pd.Timedelta(days=1)).strftime("%Y-%m-%d")

In [5]:
data

In [6]:
# query conversions
content={
     "created": {
                     "$gte": transform_date_to_timestamp_milliseconds(from_date),
                     "$lte": transform_date_to_timestamp_milliseconds(to_date),
               },
               "name":goal_name
                }
conversions=send_to_innkeepr_api_paginated(
    f"{url}/conversions/query",
    account_id,
    content,
    logging
)
conversions_all = pd.json_normalize(conversions)
conversions_all

# Do some checks

In [7]:
print(conversions_all["signal.source"].value_counts(dropna=False))
#conversions = conversions_all[conversions_all["signal.source"] == source]
conversions = conversions_all[conversions_all["sessionId"].isin(data["session"].unique())]
print(f"conversion daterange = {conversions['created'].min()} to {conversions['created'].max()}")
print(conversions["signal.source"].value_counts(dropna=False))


In [9]:
#in filtered conversions
df_max_date = data[data["created"] == data_max_date]
unique_sessions_max_date = df_max_date["session"].unique()
unique_sesions_conversions = conversions["sessionId"].unique()
print(len(unique_sesions_conversions), len(unique_sessions_max_date))
missing_sessions = [session for session in unique_sesions_conversions if session not in unique_sesions_conversions]
if len(missing_sessions) > 0:
    raise ValueError(f"Error in missing sessions: {missing_sessions}")

In [ ]:
# #in non filtered conversions
# unique_sesions_conversions_all = conversions_all["sessionId"].unique()
# print(len(unique_sesions_conversions_all), len(unique_sessions_max_date))
# missing_sessions_all = [session for session in unique_sessions_max_date if session not in unique_sesions_conversions_all]
# print(missing_sessions_all)

In [ ]:
conv_revenue = conversions["properties.revenue"].astype(float).sum().round(2)
data_revenue = df_max_date.drop_duplicates(subset=["session"])["properties.revenue"].astype(float).sum().round(2)
if conv_revenue != data_revenue:
    raise ValueError(f"Error in revenue: {conv_revenue} vs {data_revenue}")
print(f"revenue = {conv_revenue}")
print(f"revenue data = {data_revenue}")

# Implement new strategy: Conversion Value

In [ ]:
def adapt_conversion_value_to_daily_sum(df: pd.DataFrame, account_id: str):
    ignore = False #check_if_daily_conversion_sum_should_be_ignored(df, account_id)
    if ignore:
        logging.info("Ignoring daily conversion sum")
        return df
    logging.info("Adapting conversion value to daily sum")
    strategy = df["strategy"].unique()[0]
    df["properties.revenue"] = df["properties.revenue"].astype(float)
    df["value_based_conversion_value"] = df["value_based_conversion_value"].astype(
        float
    )
    df_cleaned = df.drop_duplicates(subset=["session"]).reset_index(drop=True)
    conv_sum = df_cleaned.groupby(by=["created"])["properties.revenue"].sum().round(4)
    if strategy == "synthetic_conversions":
        print(f"Use syntehtic conversions")
        df = df[df["label"] != "remove"]
        df = df.reset_index()
        count_conversions = 
    convm_sum = (
        df_cleaned.groupby(by=["created"])["value_based_conversion_value"].sum().round(4)
    )
    n = df_cleaned.groupby(by=["created"]).size()
    sum_diff = conv_sum - convm_sum
    alpha = sum_diff / n
    alpha = alpha.reset_index().rename(columns={0: "alpha"})
    print("conv_sum")
    print(conv_sum)
    print("convm_sum")
    print(convm_sum)
    print("sum_diff")
    print(sum_diff)
    print("alpha")
    print(alpha)
    print("n")
    print(n)
    # merge data
    df = pd.merge(df, alpha, how="left", left_on="created", right_on="created")
    print(df.columns)
    df["value_based_conversion_value"] = (
        df["value_based_conversion_value"] + df["alpha"]
    )
    new_convm_sum = (
        df_cleaned.groupby(by=["created"])["value_based_conversion_value"].sum().round(4)
    )
    sum_diff_new = conv_sum - new_convm_sum
    if len(sum_diff_new[sum_diff_new > 0]):
        #logging.warning(f"Error in sum_diff_new: {sum_diff_new}")
        raise ValueError(f"Error in sum_diff_new: {sum_diff_new}")
    return df


In [ ]:
data

In [ ]:
test = adapt_conversion_value_to_daily_sum(data.copy(), "")
test

In [ ]:
test["properties.revenue"].describe()

In [ ]:
test["value_based_conversion_value_new"].describe()

# Checkout data

In [ ]:
data = pd.read_csv(data_path{file_name})
data

In [ ]:
data["properties.revenue"].describe()

In [ ]:
data["value_based_conversion_value"].describe()

In [ ]:
data.groupby("created")[["properties.revenue","value_based_conversion_value"]].sum()

In [ ]:
data_before = pd.read_csv("/Users/karolinegriesbach/Documents/Innkeepr/Git/innkeepr-analytics/cate_before.csv")
data_before = data_before.rename(columns={"value_based_conversion_value": "value_based_conversion_value_before"})
data_before

In [ ]:
data_merged = pd.merge(data[["session","created","properties.revenue","value_based_conversion_value"]],data_before[["session","created","value_based_conversion_value_before"]],how="left",on=["session","created"])
data_merged[["session","created","properties.revenue","value_based_conversion_value","value_based_conversion_value_before"]]